In [3]:
import gc
import os
import pandas as pd
from glob import glob

In [22]:
sorted(glob(f"../data/interim/features/*/features*.csv"))

['../data/interim/features/app/features_2017-11-07_1700_1day_attributed.csv',
 '../data/interim/features/app/features_2017-11-08_1700_1day_attributed.csv',
 '../data/interim/features/channel/features_2017-11-07_1700_1day_attributed.csv',
 '../data/interim/features/channel/features_2017-11-08_1700_1day_attributed.csv',
 '../data/interim/features/device/features_2017-11-07_1700_1day_attributed.csv',
 '../data/interim/features/device/features_2017-11-08_1700_1day_attributed.csv',
 '../data/interim/features/os/features_2017-11-07_1700_1day_attributed.csv',
 '../data/interim/features/os/features_2017-11-08_1700_1day_attributed.csv']

In [23]:
sorted(glob(f"../data/interim/train_2017-11-0*00.csv"))

['../data/interim/train_2017-11-06_1400.csv',
 '../data/interim/train_2017-11-06_1500.csv',
 '../data/interim/train_2017-11-06_1600.csv',
 '../data/interim/train_2017-11-06_1700.csv',
 '../data/interim/train_2017-11-06_1800.csv',
 '../data/interim/train_2017-11-06_1900.csv',
 '../data/interim/train_2017-11-06_2000.csv',
 '../data/interim/train_2017-11-06_2100.csv',
 '../data/interim/train_2017-11-06_2200.csv',
 '../data/interim/train_2017-11-06_2300.csv',
 '../data/interim/train_2017-11-07_0000.csv',
 '../data/interim/train_2017-11-07_0100.csv',
 '../data/interim/train_2017-11-07_0200.csv',
 '../data/interim/train_2017-11-07_0300.csv',
 '../data/interim/train_2017-11-07_0400.csv',
 '../data/interim/train_2017-11-07_0500.csv',
 '../data/interim/train_2017-11-07_0600.csv',
 '../data/interim/train_2017-11-07_0700.csv',
 '../data/interim/train_2017-11-07_0800.csv',
 '../data/interim/train_2017-11-07_0900.csv',
 '../data/interim/train_2017-11-07_1000.csv',
 '../data/interim/train_2017-11-07

In [2]:
dtypes = {
    'ip': 'uint32',
    'app': 'uint16',
    'device': 'uint16',
    'os': 'uint16',
    'channel': 'uint16',
    'is_attributed': 'uint8'
}
to_read = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']

target_entities = ['app', 'device', 'os', 'channel']

In [3]:
train_files = [
    '../data/interim/train_2017-11-09_0400.csv',
    '../data/interim/train_2017-11-09_0500.csv',
    '../data/interim/train_2017-11-09_0600.csv',
]

In [4]:
for train_file in train_files:
    df_train = pd.read_csv(train_file, usecols=to_read, dtype=dtypes)
    name, ext = os.path.splitext(os.path.basename(train_file))
    for target_entity in target_entities:
        feature_files = sorted(glob(f"../data/interim/features/{target_entity}/features_2017-11-08_1700*.csv"))
        assert len(feature_files) > 0
        for feature_file in feature_files:
            df_feature = pd.read_csv(feature_file)
            df_feature = df_feature.set_index(target_entity)
            df_train = df_train.join(df_feature, on=target_entity)
            del df_feature
    
    filename_output = f"../data/interim/combined/{name}_attributed.hdf.compress"
    print(f"Writing {filename_output}")
    df_train.to_hdf(filename_output, 'train', mode='w', complib='blosc', fletcher32=True)
    del df_train
    gc.collect()

Writing ../data/interim/combined/train_2017-11-09_0400_attributed.hdf.compress
Writing ../data/interim/combined/train_2017-11-09_0500_attributed.hdf.compress
Writing ../data/interim/combined/train_2017-11-09_0600_attributed.hdf.compress


In [29]:
%%time
df = pd.read_hdf('../data/interim/combined/train_2017-11-07_1700_attributed.hdf.compress')

CPU times: user 24 ms, sys: 429 ms, total: 453 ms
Wall time: 449 ms


In [30]:
df.head()

,ip,app,device,os,channel,is_attributed,PERCENT_TRUE(clicks.is_attributed)_app_1day,AVG_TIME_BETWEEN(clicks.click_time)_app_1day,MODE(clicks.ip)_app_1day,MODE(clicks.device)_app_1day,...,MODE(clicks.os)_channel_1day,COUNT(clicks)_channel_1day,AVG_TIME_BETWEEN(clicks.click_time WHERE is_attributed = True)_channel_1day,MODE(clicks.ip WHERE is_attributed = True)_channel_1day,MODE(clicks.app WHERE is_attributed = True)_channel_1day,MODE(clicks.device WHERE is_attributed = True)_channel_1day,MODE(clicks.os WHERE is_attributed = True)_channel_1day,MODE(clicks.HOUR(click_time))_channel_1day,MODE(clicks.HOUR(click_time) WHERE is_attributed = True)_channel_1day,COUNT(clicks WHERE is_attributed = True)_channel_1day
0,106598,12,1,18,178,0,0.000114,0.010638,73516.0,1.0,...,13.0,1973266,345.466667,5314.0,12.0,1.0,19.0,9.0,4.0,241
1,7775,12,1,17,178,0,0.000114,0.010638,73516.0,1.0,...,13.0,1973266,345.466667,5314.0,12.0,1.0,19.0,9.0,4.0,241
2,73487,21,1,19,232,0,0.000133,0.076109,5348.0,1.0,...,19.0,444575,162.467043,5348.0,9.0,1.0,19.0,16.0,1.0,532
3,140670,8,1,23,145,0,0.001784,0.069916,5348.0,1.0,...,19.0,1114908,57.864430,5348.0,8.0,1.0,19.0,16.0,0.0,1491
4,7393,12,1,20,265,0,0.000114,0.010638,73516.0,1.0,...,13.0,1722129,391.045455,100275.0,12.0,1.0,19.0,10.0,5.0,221


In [6]:
sorted(glob(f"../data/interim/combined/train_2017-11-09*.hdf.compress"))

['../data/interim/combined/train_2017-11-09_0400_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-09_0500_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-09_0600_attributed.hdf.compress']

In [1]:
train_feature_files = ['../data/interim/combined/train_2017-11-09_0400_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-09_0500_attributed.hdf.compress',
 '../data/interim/combined/train_2017-11-09_0600_attributed.hdf.compress']

In [4]:
df_train = pd.concat([pd.read_hdf(f) for f in train_feature_files], ignore_index=True)

In [5]:
df_train.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed',
       'PERCENT_TRUE(clicks.is_attributed)_app_1day',
       'AVG_TIME_BETWEEN(clicks.click_time)_app_1day',
       'MODE(clicks.ip)_app_1day', 'MODE(clicks.device)_app_1day',
       'MODE(clicks.os)_app_1day', 'MODE(clicks.channel)_app_1day',
       'COUNT(clicks)_app_1day',
       'AVG_TIME_BETWEEN(clicks.click_time WHERE is_attributed = True)_app_1day',
       'MODE(clicks.ip WHERE is_attributed = True)_app_1day',
       'MODE(clicks.device WHERE is_attributed = True)_app_1day',
       'MODE(clicks.os WHERE is_attributed = True)_app_1day',
       'MODE(clicks.channel WHERE is_attributed = True)_app_1day',
       'MODE(clicks.HOUR(click_time))_app_1day',
       'MODE(clicks.HOUR(click_time) WHERE is_attributed = True)_app_1day',
       'COUNT(clicks WHERE is_attributed = True)_app_1day',
       'PERCENT_TRUE(clicks.is_attributed)_device_1day',
       'AVG_TIME_BETWEEN(clicks.click_time)_device_1day',
      

In [6]:
df_train.to_hdf('../data/processed/test_2017-11-09_0400_09_0600_attributed.hdf.compress', 'test', mode='w', complib='blosc', fletcher32=True)